# Importação de base de dados

In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
import sklearn.metrics as metrics

from sklearn.ensemble import RandomForestClassifier

from keras.models import Sequential
from keras.layers import Dense, InputLayer

2021-09-05 15:39:13.312596: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-05 15:39:13.312701: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [18]:
transact_train_basic = pd.read_csv('transact_train_database_x.csv')
transact_test_basic = pd.read_csv('transact_test_database_x.csv')

transact_train_basic_y = pd.read_csv('transact_train_database_y.csv')

In [19]:
transact_train_basic.head()

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,onlineStatus,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder
0,1,6,5,39.887,1,59.99000,59.990000,59.990000,1,59.990000,...,y,completely orderable,1,600.000000,70.000000,21.000000,1.000000,43.000000,1.000000,49.000000
1,2,6,5,94.469,0,72.11816,157.646205,1096.642791,0,86.444819,...,y,completely orderable,?,2266.582859,483.731568,128.169304,12.893535,44.921048,1.706919,79.007675
2,3,6,5,341.613,11,9.99000,29.990000,109.950000,2,9.990000,...,y,completely orderable,3,1800.000000,475.000000,302.000000,12.000000,45.000000,1.000000,11.000000
3,4,6,5,42.812,4,4.99000,4.990000,19.960000,1,4.990000,...,y,completely not orderable,?,2266.582859,483.731568,128.169304,12.893535,44.921048,1.706919,79.007675
4,5,6,5,2816.046,45,12.99000,179.950000,1093.720000,4,19.990000,...,y,completely orderable,4,800.000000,503.000000,18.000000,1.000000,46.000000,1.000000,40.000000


# Classificador Random Forest

In [20]:
rc_basic = RandomForestClassifier(n_estimators=100, max_features=10)
rc_basic.fit(transact_train_basic, transact_train_basic_y.order.to_numpy())

ValueError: could not convert string to float: 'y'